*Extracción de datos de LinkedIn V2 (Con resumen de la empresa)*

Cargando librerías

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException,ElementClickInterceptedException, TimeoutException
import pandas as pd
import time

Iniciando el driver

In [4]:
#abriendo web driver
driver= webdriver.Chrome()

#especificando url
url_login='https://www.linkedin.com/'

#abriendo url
driver.get(url_login)

Iniciando sesión

In [5]:
#Encontrando elementos de inicio de sesion
username_field = driver.find_element(By.ID, "session_key")
password_field = driver.find_element(By.ID, "session_password")
login_button = driver.find_element(By.XPATH, "//button[@type='submit']")

# iniciando sesión
username_field.send_keys("scselenium123@gmail.com")
password_field.send_keys("mercurio123")


# Clickeando el botón de inicio de sesión
login_button.click()

Obteniendo urls de empresas

In [6]:
datos_empresas = [] #creando espacio para guardar información
urls= [] #creando espacio para almacenar urls

for n in range(1,3): #Cambiar el rango acorde al número de páginas
    string= f'https://www.linkedin.com/search/results/companies/?companyHqGeo=%5B%22101526486%22%2C%22106106375%22%2C%22105047677%22%5D&companySize=%5B%22B%22%2C%22C%22%5D&origin=FACETED_SEARCH&page={n}&sid=NLw'
    urls.append(string)

print(urls) 

['https://www.linkedin.com/search/results/companies/?companyHqGeo=%5B%22101526486%22%2C%22106106375%22%2C%22105047677%22%5D&companySize=%5B%22B%22%2C%22C%22%5D&origin=FACETED_SEARCH&page=1&sid=NLw', 'https://www.linkedin.com/search/results/companies/?companyHqGeo=%5B%22101526486%22%2C%22106106375%22%2C%22105047677%22%5D&companySize=%5B%22B%22%2C%22C%22%5D&origin=FACETED_SEARCH&page=2&sid=NLw']


Extrayendo los url /about de las empresas

In [7]:
enlaces_empresas = []
for url in urls:
    driver.get(url)
# Espera hasta que el contenedor principal de los resultados esté presente
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".reusable-search__entity-result-list")))
# Encuentra todos los elementos <li> que representan cada empresa dentro del contenedor principal
    contenedores_empresas = driver.find_elements(By.CSS_SELECTOR, ".reusable-search__entity-result-list > li.reusable-search__result-container")
    for contenedor in contenedores_empresas:
    # Dentro de cada contenedor <li>, encuentra el elemento <a> y extrae el atributo 'href'
        enlace = contenedor.find_element(By.CSS_SELECTOR, 'a.app-aware-link').get_attribute('href')
        enlace_about= f'{enlace}about/'
        enlaces_empresas.append(enlace_about)

# Imprime los enlaces de las empresas
for enlace_about in enlaces_empresas:
    print(enlace_about)


https://www.linkedin.com/company/jobs-empleo/about/
https://www.linkedin.com/company/el-ceo/about/
https://www.linkedin.com/company/la-head-hunter/about/
https://www.linkedin.com/company/isproxwork/about/
https://www.linkedin.com/company/expertos-en-competencias/about/
https://www.linkedin.com/company/predictiva21/about/
https://www.linkedin.com/company/forbes-centroam%C3%A9rica/about/
https://www.linkedin.com/company/de-codigo-somos/about/
https://www.linkedin.com/company/codigofacilito/about/
https://www.linkedin.com/company/vacantesdigitales/about/
https://www.linkedin.com/company/rh-en-las-empresas/about/
https://www.linkedin.com/company/codigofacilito/about/
https://www.linkedin.com/company/itthhgroup/about/
https://www.linkedin.com/company/meetingdoctors/about/
https://www.linkedin.com/company/hirelineio/about/
https://www.linkedin.com/company/be-wanted/about/
https://www.linkedin.com/company/igeneris/about/
https://www.linkedin.com/company/soy-startup/about/
https://www.linkedin

Extrae los datos

In [8]:
for enlace_about in enlaces_empresas:
    driver.get(enlace_about)
    time.sleep(5)  # Espera para asegurar que la página ha cargado completamente.
    
    # Espera hasta que sea visible un elemento identificativo de la página.
    wait = WebDriverWait(driver, 10)
    wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'p.t-black--light')))
    
    # Intenta capturar el nombre.
    try:
        nombre = driver.find_element(By.CSS_SELECTOR, 'h1.org-top-card-summary__title').text
    except NoSuchElementException:
        pass  # Si no se encuentra, se mantiene "Nombre no encontrado"
    
    # Intenta capturar la descripción.
    try:
        descripcion = driver.find_element(By.CSS_SELECTOR, '.org-top-card-summary__tagline').text
    except NoSuchElementException:
        pass  # Si no se encuentra, se mantiene "Descripción no encontrada"
     
    #Intenta capturar el tipo de negocio, sede, número de seguidores y número de empleados
    try: 
        elements = driver.find_elements(By.CSS_SELECTOR, ".org-top-card-summary-info-list__info-item, .org-top-card-summary-info-list__info-item a")
        if elements:
            tipo_negocio = elements[0].text if len(elements) > 0 else "Tipo de negocio no disponible"
            sede= elements[1].text if len(elements)> 0 else 'Sede no disponible'
            numero_seguidores = elements[2].text if len(elements) > 2 else "Número de seguidores no disponible"
            numero_empleados = elements[3].text if len(elements) > 3 else "Número de empleados no disponible"
    except NoSuchElementException:
        pass
    #Intenta capturar el resumen
    try:
        resumen = driver.find_element(By.CSS_SELECTOR, 'p.break-words.white-space-pre-wrap.t-black--light.text-body-medium').text
    except NoSuchElementException:
        resumen = "Descripción no encontrada"
    
    #Intenta capturar el sitio web
    
    ## Encuentra todos los elementos <dt> y <dd> dentro del contenedor <dl>.
    dt_elements = driver.find_elements(By.CSS_SELECTOR, "dl.overflow-hidden > dt")
    dd_elements = driver.find_elements(By.CSS_SELECTOR, "dl.overflow-hidden > dd")
    
    ## Itera a través de los elementos <dt> para determinar qué información sigue a cada título.
    for i, dt in enumerate(dt_elements):
        titulo = dt.text.strip()
        if i < len(dd_elements):  # Asegura que no se exceda el índice.
            descripcion_dd = dd_elements[i].text.strip()
            if "Sitio web" in titulo:
                sitio_web = descripcion_dd
    
    # Imprime la información recopilada para cada empresa.
    print(f"Nombre: {nombre}")
    print(f"Descripción: {descripcion}")
    print(f"Sector: {tipo_negocio}")
    print(f'Sede: {sede}')
    print(f"Tamaño de la empresa: {numero_empleados}")
    print(f'Seguidores: {numero_seguidores}')
    print(f'Sitio web:{sitio_web}')
    print(f'Resumen: {resumen}')
    print("-----------------------------------------------------")
    
    datos_empresas.append([nombre, descripcion, tipo_negocio, sede, numero_seguidores, numero_empleados, enlace, resumen])

Nombre: JOB`S EMPLEO
Descripción: Las Herramientas que te cambiaran la vida están aquí
Sector: Servicios de recursos humanos
Sede: cd de mexico, cd de mexico
Tamaño de la empresa: 2-10 empleados
Seguidores: 426 mil seguidores
Sitio web:https://jobs-empleo.com/
Resumen: JOB'S EMPLEO es el espacio donde puedes ver las vacantes que te cambiaran la vida.

Si deseas poder acceder al entrenamiento de entrevistas lo que tienes que hacer es darte de alta en la pagina después ve a la publicación que esta fija y sigue las instrucciones.
-----------------------------------------------------
Nombre: EL CEO
Descripción: Periodismo digital de negocios.
Sector: Medios de audio y vídeo en línea
Sede: Santa Fe, Federal District
Tamaño de la empresa: 11-50 empleados
Seguidores: 259 mil seguidores
Sitio web:http://elceo.com/
Resumen: Periodismo digital de negocios. Somos #ELCEO contacto@elceo.com
-----------------------------------------------------
Nombre: L.A. Head Hunter
Descripción: Brindamos solucio

Almacena los datos en un csv

In [9]:

df_empresas = pd.DataFrame(datos_empresas, columns=['Nombre de la Empresa', 'Descripción', 'Tipo de Negocio', 'sede', 'Número de Seguidores', 'Número de Empleados', 'Link de la Empresa', 'resumen'])

# Guardando los datos en un archivo CSV
df_empresas.to_csv('datos_empresas_linkedin_v2.csv', index=False)

# Cerrando el navegador
driver.quit()

# Mostrar el DataFrame
print(df_empresas)

           Nombre de la Empresa  \
0  Worldwide Recruitment Energy   

                                         Descripción  \
0  Ayudamos en la búsqueda de profesionales técni...   

                    Tipo de Negocio Número de Seguidores Número de Empleados  \
0  Dotación y selección de personal     9 mil seguidores      2-10 empleados   

                            Link de la Empresa  
0  https://www.linkedin.com/company/wwrenergy/  
